In [ ]:
# put yolov7_pitch.yaml to cfg/training folder
# pitch_data.yaml to data folder

## training is launched via command line with command:
### python train.py --workers 1 --device 0 --batch-size 8 --epochs 100 --img 640 640 --data data/pitch_data.yaml --hyp data/hyp.scratch.custom.yaml --cfg cfg/training/yolov7_pitch.yaml --name yolov7_pitch_0 --weights yolov7.pt

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
import shutil
from PIL import Image, ImageDraw
import numpy as np

In [ ]:
path = os.getcwd()

In [ ]:
path

In [ ]:
# results from 1. training are located at:
# ./runs/train/yolov7_pitch_04
# best weights named best_0 are placed in ./

## running inference on test data

In [ ]:
!cd .

In [ ]:
# put your path to main directory here
os.chdir('C:\\Users\\Kossa\\Desktop\\respo_vision\\yolov7-pitch')

In [ ]:
os.getcwd()

In [ ]:
!python detect.py --weights best_0.pt --conf 0.5 --img-size 640 --source inference/images --save-txt 

In [ ]:
os.chdir('C:\\Users\\Kossa\\Desktop\\respo_vision\\yolov7-pitch\\notebook')

In [ ]:
os.getcwd()

In [ ]:
vis_val_df = pd.read_csv('vis_val_df_0.csv').drop('Unnamed: 0', axis = 1 )

In [ ]:
val_df = pd.read_csv('val_df_0.csv').drop('Unnamed: 0', axis = 1 )

In [ ]:
val_df

In [ ]:
val_df.isna().any().any()

In [ ]:
path2 = 'C:\\Users\\Kossa\\Desktop\\respo_vision\\yolov7-pitch'
#yolov7-pitch\runs\detect\exp\labels <---- location of results labels

In [ ]:
no_vis_classes_val = ['a37d3123f9202186d20a7765ec2c0a.jpg',
 '934a5454a825420abe74ace1cbe2b0.jpg',
 'fffe2a9e755bf705c79753bf09b7f1.jpg',
 '73aec0a4b73935433808780a577f3c.jpg',
 '32e27114c613d5a58cc95b3f6262ab.jpg',
 '2e031ce6ec3a9b7a45d70bcdf7618e.jpg',
 '03a17019ec63792e8316633d50d109.jpg',
 '3e6cbfd6d83cc24393092a74522062.jpg',
 '07b7b54074da586b6c0effd7f5ba9e.jpg',
 '5ad1adfad421cb8bd6ced67369cd38.jpg',
 '370bcfad2192293a0a25daf431d7d4.jpg',
 'ae1ba09379c7277fa88c5cff12c398.jpg',
 '7db8876376346a856b26cf799e97dd.jpg']

In [ ]:
no_vis_classes_val = [item[:-4] for item in no_vis_classes_val]

In [ ]:
no_vis_classes_val

In [ ]:
# yolo format - object-class-id> <x> <y> <width> <height>
#filling val_df with predictions - x_pred and y_pred

infer_data = {}
for filename in val_df['image_path'].unique():

    # print(df[1]['x'])
    source_path = filename
    filename = filename.split('/')[-1][:-4]

    if filename in no_vis_classes_val:
        print(filename)
        continue
    # print(filename, '\n')
    temp = []
    duplicate_classes = [] #checking for duplciate classes in output

    with open(f'{path2}\\runs\\detect\\exp\\labels\\{filename}.txt', 'r') as file:
        lines = file.readlines()
        for line in lines:
            # print(line)
            data = line.split(' ')
            if data[0] not in temp:
                temp.append(data[0])
            else:
                duplicate_classes.append(data[0])

            val_df.loc[(val_df['image_path'] == source_path) & (val_df['kid'] == int(data[0])), 
                        ['x_pred', 'y_pred']] = int(float(data[1])*1920), int(float(data[2])*1080)
                                                                                                                        
                

In [ ]:
duplicate_classes

In [ ]:
# That is very good. the model did not predict the same class twice for one image. 
# It is quite obvious, since training data never contained 2 classes at once, but it was good to confirm this
# That simplifies the analysis and I can just input predictions to existing df

In [ ]:
# checking filenames with confusions

# <object-class-id> <x> <y> <width> <height>
# label_pad = 50 #px
confused_list = []
for filename in vis_val_df['image_path'].unique():
    # print(df[1]['x'])
    source_path = filename
    filename = filename.split('/')[-1][:-4]
    # print(filename, '\n')

    duplicate_classes = [] #checking for duplciate classes in output

    # print(f'************************* {filename} ***********************')
    
    temp = []
    with open(f'{path2}\\runs\\detect\\exp\\labels\\{filename}.txt', 'r') as file:
        lines = file.readlines()
        for line in lines:
            data = line.split(' ')
            temp.append(int(data[0]))
        # print(sorted(temp))

    temp2 = []    
    with open(f'{path2}\\data\\val\\labels\\{filename}.txt', 'r') as file:
        lines = file.readlines()
        for line in lines:
            data = line.split(' ')
            temp2.append(int(data[0]))
        # print(sorted(temp))

    if sorted(temp) != sorted(temp2):
        print(filename)
        print(sorted(temp))
        print(sorted(temp2))
        confused_list.append(filename)
                



                                                                                                                        


In [ ]:
# for plotting

class_name_to_id_mapping = {str(i):i for i in range(40)}

class_id_to_name_mapping = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))

def plot_bounding_box(image, annotation_list):
    annotations = np.array(annotation_list)
    w, h = image.size
    
    plotted_image = ImageDraw.Draw(image)

    transformed_annotations = np.copy(annotations)
    transformed_annotations[:,[1,3]] = annotations[:,[1,3]] * w
    transformed_annotations[:,[2,4]] = annotations[:,[2,4]] * h 
    
    transformed_annotations[:,1] = transformed_annotations[:,1] - (transformed_annotations[:,3] / 2)
    transformed_annotations[:,2] = transformed_annotations[:,2] - (transformed_annotations[:,4] / 2)
    transformed_annotations[:,3] = transformed_annotations[:,1] + transformed_annotations[:,3]
    transformed_annotations[:,4] = transformed_annotations[:,2] + transformed_annotations[:,4]
    
    for ann in transformed_annotations:
        obj_cls, x0, y0, x1, y1 = ann
        plotted_image.rectangle(((x0,y0), (x1,y1)))
        
        plotted_image.text((x0, y0 - 10), class_id_to_name_mapping[(int(obj_cls))])
    
    plt.imshow(np.array(image))
    plt.show()

In [ ]:
path2

In [ ]:
# filling val_df with predictions for confused preds
for filename in confused_list:

    temp = []
    temp2 = []

    print(filename)

    img_path = f'{path2}\\data\\val\\images\\{filename}.jpg'
    labels_path = f'{path2}\\data\\val\\labels\\{filename}.txt'
    
    img_path_infer = f'{path2}\\runs\\detect\\exp\\{filename}.jpg'
    labels_path_infer = f'{path2}\\runs\\detect\\exp\\labels\\{filename}.txt'
    
    image = Image.open(img_path)
    with open(labels_path, 'r') as file:
        labels = file.readlines()

    for line in labels:
        data = line.split(' ')
        temp.append(int(data[0]))

    print(sorted(temp))
    
    annotation_file = labels_path
    with open(annotation_file, "r") as file:
        annotation_list = file.read().split("\n")[:-1]
        annotation_list = [x.split(" ") for x in annotation_list]
        annotation_list = [[float(y) for y in x ] for x in annotation_list]
        
    plot_bounding_box(image, annotation_list)

    image = Image.open(img_path_infer)
    with open(labels_path_infer, 'r') as file:
        labels = file.readlines()

    for line in labels:
        data = line.split(' ')
        temp2.append(int(data[0]))

    print(sorted(temp2))
    
    annotation_file = labels_path
    with open(annotation_file, "r") as file:
        annotation_list = file.read().split("\n")[:-1]
        annotation_list = [x.split(" ") for x in annotation_list]
        annotation_list = [[float(y) for y in x ] for x in annotation_list]
        
    plot_bounding_box(image, annotation_list)

In [ ]:
# Some EDA and checks for confused files
confused_list

In [ ]:
confused_list_path = [f'images/train/{filename}.jpg' for filename in confused_list]

In [ ]:
val_df.head(60)

In [ ]:
val_df.fillna(0, inplace = True)
val_df

In [ ]:
pd.set_option('display.max_rows', 100)
val_df[val_df['image_path'].isin(confused_list_path)].head(100)

In [ ]:
val_df.loc[(val_df['x_pred'] > 0) | (val_df['y_pred'] > 0)  ,'vis_pred'] = 2

In [ ]:
val_df.fillna(0, inplace = True)

In [ ]:
# introducing correct location - corr_loc column

corr_loc = []
for row in val_df.itertuples():
    if row.vis == row.vis_pred == 2 and abs(row.x - row.x_pred) < 50 and abs(row.y - row.y_pred) < 50:
        corr_loc.append(1)
    elif row.vis == row.vis_pred == 0:
        corr_loc.append(1)
    else:
        corr_loc.append(0)
val_df['corr_loc'] = corr_loc

In [ ]:
val_df.head(20)

In [ ]:
val_df[val_df['corr_loc'] == 0]

In [ ]:
# comapring corr_loc with files with confusions

print(len(val_df[val_df['corr_loc'] == 0]['image_path'].unique()))

len(confused_list)
# val_df[val_df['corr_loc'] == 0]['image_path'].unique()
# almost agrees with the list

In [ ]:
diff_confused = [item for item in val_df[val_df['corr_loc'] == 0]['image_path'].unique() if item not in confused_list_path]

In [ ]:
diff_confused

In [ ]:
diff_confused_2 = [item for item in confused_list_path if item not in  val_df[val_df['corr_loc'] == 0]['image_path'].unique()]

In [ ]:
diff_confused_2

In [ ]:
# classes are correct, but distance is too high
val_df[val_df['image_path'].isin(diff_confused)].head(100)

In [ ]:
#that one is actually correct
val_df[val_df['image_path'].isin(diff_confused_2)] 

In [ ]:
val_df[val_df['image_path'].isin(diff_confused)].head(100)

In [ ]:
# determining prediction of kid. - 0 if it fits nowhere

conf_pred_kid = []
for row in val_df.itertuples():
    if not row.corr_loc:
        _flag = True
        temp_df = val_df[val_df['image_path'] == row.image_path]
        for row2 in val_df.itertuples():
            if abs(row.x_pred - row2.x) < 50 and abs(row.y_pred - row.y) < 50:
                conf_pred_kid.append(row2.kid)
                _flag = False
                break
        if _flag:
            conf_pred_kid.append(0)
    else:
        conf_pred_kid.append(row.kid)

val_df['pred_kid'] = conf_pred_kid

In [ ]:
val_df[val_df['image_path'].isin(confused_list_path)].head(100)

In [ ]:
val_df[val_df['kid']!=val_df['pred_kid']]#.head(100)

In [ ]:
val_df[(val_df['kid']!=val_df['pred_kid']) & val_df['pred_kid']]#.head(100)

In [ ]:
val_df[val_df['image_path'].isin(confused_list_path)].head(100)

## calculating metrices

## Avg for all classes

In [ ]:
# time to remove it
fake_kid = [24,36,37,38]

In [ ]:
val_df = val_df[~val_df['kid'].isin(fake_kid)]

In [ ]:
val_df

In [ ]:
val_df.loc[ (val_df['corr_loc'] == 1) & (val_df['vis'] == 2) & (val_df['vis_pred'] == 2) ,'outcome'] = 'TP'
val_df.loc[ (val_df['corr_loc'] == 1) & (val_df['vis'] == 0) & (val_df['vis_pred'] == 0) ,'outcome'] = 'TN'
val_df.loc[ (val_df['corr_loc'] == 0) & (val_df['vis'] == 0) & (val_df['vis_pred'] == 2) ,'outcome'] = 'FP'
val_df.loc[ (val_df['corr_loc'] == 0) & (val_df['vis'] == 2) & (val_df['vis_pred'] == 0) ,'outcome'] = 'FN'

In [ ]:
val_df.loc[val_df['outcome'].isna()]

In [ ]:
val_df

In [ ]:
val_df.loc[ (val_df['corr_loc'] == 0) & (val_df['vis'] == 2) & (val_df['vis_pred'] == 2) ,'outcome'] = 'CONF'

In [ ]:
val_df.columns

In [ ]:
# check if CONT is correct
val_df.loc[val_df['outcome'] == 'CONF', ['x', 'y', 'vis', 'kid', 'dataset', 'image_path', 'x_pred', 'y_pred',
       'vis_pred', 'corr_loc', 'pred_kid']]

In [ ]:
val_df.head(100)#.loc[val_df['outcome'].isna()]

In [ ]:
grouped_df = val_df.groupby('outcome').count()
grouped_df

In [ ]:
accuracy = (grouped_df.loc['TP', 'x'] + grouped_df.loc['TN', 'x']) / ( grouped_df.loc['FP', 'x'] + grouped_df.loc['FN', 'x'] + grouped_df.loc['TP', 'x'] + grouped_df.loc['TN', 'x'])
precision = grouped_df.loc['TP', 'x'] / ( grouped_df.loc['TP', 'x'] + grouped_df.loc['FP', 'x'])
recall = grouped_df.loc['TP', 'x'] / ( grouped_df.loc['TP', 'x'] + grouped_df.loc['FN', 'x'])
f1 = 2* precision * recall / ( precision + recall )
info = recall - 1 + grouped_df.loc['TN', 'x'] / ( grouped_df.loc['TN', 'x'] + grouped_df.loc['FP', 'x'])
mark = precision - 1 + grouped_df.loc['TN', 'x'] / ( grouped_df.loc['TN', 'x'] + grouped_df.loc['FN', 'x'])
mcc = ((info * mark)**0.5)
conf_rate = grouped_df.loc['CONF', 'x'] / ( grouped_df.loc['FP', 'x'] + grouped_df.loc['FN', 'x'] + grouped_df.loc['TP', 'x'])
conf_among_mistakes = grouped_df.loc['CONF', 'x'] / ( grouped_df.loc['FP', 'x'] + grouped_df.loc['FN', 'x'])

In [ ]:
print(accuracy)
print(precision)
print(recall)
print(f1)
print(info)
print(mark)
print(mcc)
print(conf_rate)
print(conf_among_mistakes)

## per class

In [ ]:
df_metrics_per_kid = pd.DataFrame()
for tup in val_df.groupby('kid'):
    df_kid = tup[1]
    df_kid = df_kid.groupby('outcome').count()
    # print(df_kid)
    for key in ['FN', 'FP', 'TN', 'TP', 'CONF']:
        if key not in df_kid.index:
            df_kid.loc[key, 'x'] = 0
    print(df_kid)
    accuracy = (df_kid.loc['TP', 'x'] + df_kid.loc['TN', 'x']) / ( df_kid.loc['FP', 'x'] + df_kid.loc['FN', 'x'] + df_kid.loc['TP', 'x'] + df_kid.loc['TN', 'x'])
    precision = df_kid.loc['TP', 'x'] / ( df_kid.loc['TP', 'x'] + df_kid.loc['FP', 'x'])
    recall = df_kid.loc['TP', 'x'] / ( df_kid.loc['TP', 'x'] + df_kid.loc['FN', 'x'])
    f1 = 2* precision * recall / ( precision + recall )
    info = recall - 1 + df_kid.loc['TN', 'x'] / ( df_kid.loc['TN', 'x'] + df_kid.loc['FP', 'x'])
    mark = precision - 1 + df_kid.loc['TN', 'x'] / ( df_kid.loc['TN', 'x'] + df_kid.loc['FN', 'x'])
    mcc = ((info * mark)**0.5)
    conf_rate = df_kid.loc['CONF', 'x'] / ( df_kid.loc['FP', 'x'] + df_kid.loc['FN', 'x'] + df_kid.loc['TP', 'x'])
    conf_among_mistakes = df_kid.loc['CONF', 'x'] / ( df_kid.loc['FP', 'x'] + df_kid.loc['FN', 'x'])
    df_metrics_per_kid.loc[tup[0], ['accuracy', 'precision', 'recall', 'f1', 'informedness', 'markedness', 'MCC', 'CONF_rate', 'CONF_per_F']] = accuracy, precision, recall, f1, info, mark, mcc, conf_rate, conf_among_mistakes

In [ ]:
df_metrics_per_kid

In [ ]:
# fig, ax = plt.subplots()
# fig = plt.Figure(fig_size = (8, 16))
# ax.barh(df_metrics_per_kid.index, df_metrics_per_kid[['f1', 'MCC']])
# ax.barh(df_metrics_per_kid.index, df_metrics_per_kid[])
# Add some text for labels, title and custom x-axis tick labels, etc.

ax = df_metrics_per_kid[['f1', 'MCC']].plot.barh(figsize=(4,8))
ax.set_ylabel('Keypoint')
ax.set_title('Key metrics per class')
# ax.set_yticks(labels=[str(index) for index in df_metrics_per_kid.index])

# x.set_ylim(8, 16)

plt.show()
plt.close()
ax = df_metrics_per_kid[['precision', 'recall']].plot.barh(figsize=(4,8), color = ['green','purple'])
ax.set_ylabel('Keypoint')
ax.set_title('Key metrics per class')
# ax.set_yticks(labels=[str(index) for index in df_metrics_per_kid.index])

# x.set_ylim(8, 16)

plt.show()
plt.close()

## metrics for class placement precision - MAE and MSE are suitable

In [ ]:
# we can apporach it in the soft way and in the hard way
# soft - we will not penalize model for confusing classes and FP and FN. Therefore we will only calculate AE and SE for TP. TN will be infored for obvious reasons.
# medium - we will penalize model for confusing classes - TP and CONF
# hard - we will penalize model for everything - FP, FN and CONF classes

In [ ]:
val_df

In [ ]:
val_df.loc[val_df['outcome'] == 'TP',  'AEx_soft'] = abs(val_df['x'] - val_df['x_pred'])
val_df.loc[val_df['outcome'] == 'TP',  'AEy_soft'] = abs(val_df['y'] - val_df['y_pred'])
val_df.loc[(val_df['outcome'] == 'TP') | (val_df['outcome'] == 'CONF'),  'AEx_med'] = abs(val_df['x'] - val_df['x_pred'])
val_df.loc[(val_df['outcome'] == 'TP') | (val_df['outcome'] == 'CONF'),  'AEy_med'] = abs(val_df['y'] - val_df['y_pred'])
val_df.loc[val_df['outcome'] != 'TN',  'AEx_hard'] = abs(val_df['x'] - val_df['x_pred'])
val_df.loc[val_df['outcome'] != 'TN',  'AEy_hard'] = abs(val_df['y'] - val_df['y_pred'])


In [ ]:
val_df

In [ ]:
val_df['AE_soft'] = (val_df['AEx_soft']**2 + val_df['AEy_soft']**2)**0.5
val_df['AE_med'] = (val_df['AEx_med']**2 + val_df['AEy_med']**2)**0.5
val_df['AE_hard'] = (val_df['AEx_hard']**2 + val_df['AEy_hard']**2)**0.5

In [ ]:
val_df.loc[val_df['outcome'] == 'TP',  'SEx_soft'] = (val_df['x'] - val_df['x_pred'])**2
val_df.loc[val_df['outcome'] == 'TP',  'SEy_soft'] = (val_df['y'] - val_df['y_pred'])**2
val_df.loc[(val_df['outcome'] == 'TP') | (val_df['outcome'] == 'CONF'),  'SEx_med'] = (val_df['x'] - val_df['x_pred'])**2
val_df.loc[(val_df['outcome'] == 'TP') | (val_df['outcome'] == 'CONF'),  'SEy_med'] = (val_df['y'] - val_df['y_pred'])**2
val_df.loc[val_df['outcome'] != 'TN',  'SEx_hard'] = (val_df['x'] - val_df['x_pred'])**2
val_df.loc[val_df['outcome'] != 'TN',  'SEy_hard'] = (val_df['y'] - val_df['y_pred'])**2


In [ ]:
val_df['SE_soft'] = val_df['SEx_soft'] + val_df['AEy_soft']
val_df['SE_med'] = val_df['SEx_med'] + val_df['AEy_med']
val_df['SE_hard'] = val_df['SEx_hard'] + val_df['AEy_hard']

In [ ]:
val_df

In [ ]:
for err in ['AE_soft', 'AE_med', 'AE_hard', 'SE_soft', 'SE_med', 'SE_hard']:
    print(f'M{err}: ', val_df[err].mean())

## average per class

In [ ]:
for err in ['AE_soft', 'AE_med', 'AE_hard', 'SE_soft', 'SE_med', 'SE_hard']:

    print(f'M{err}: ', val_df[err].mean())

In [ ]:
df_metrics_per_kid_2 = val_df[['kid','AE_soft', 'AE_med', 'AE_hard', 'SE_soft', 'SE_med', 'SE_hard']].groupby('kid').mean()
df_metrics_per_kid_2 = df_metrics_per_kid_2.rename(columns={"AE_soft": "MAE_soft", "SE_soft": "MSE_soft"})

In [ ]:
df_metrics_per_kid_2 

In [ ]:
# fig, ax = plt.subplots()
# fig = plt.Figure(fig_size = (8, 16))
# ax.barh(df_metrics_per_kid.index, df_metrics_per_kid[['f1', 'MCC']])
# ax.barh(df_metrics_per_kid.index, df_metrics_per_kid[])
# Add some text for labels, title and custom x-axis tick labels, etc.

ax = df_metrics_per_kid_2[['MAE_soft', 'MSE_soft']].plot.barh(figsize=(4,8))
ax.set_ylabel('Keypoint')
ax.set_title('Key metrics per class')
# ax.set_yticks(labels=[str(index) for index in df_metrics_per_kid.index])

# x.set_ylim(8, 16)

plt.show()
plt.close()


In [ ]:
val_df

In [ ]:
val_df[(val_df['kid'] == 3) &(val_df['vis'] == 2)]

In [ ]:
val_df[(val_df['kid'] == 23) &(val_df['vis'] == 2)]

In [ ]:
val_df[(val_df['kid'] == 30) &(val_df['vis'] == 2)]

In [ ]:
# the model is working very well even for rare classes, but we do not have too much cases to test... Augmentation is needed